### Wczytać graf Facebooka, krawędzie z pliku musae_facebook_edges.csv, atrybuty page_name oraz page_type z pliku musae_facebook_target.csv.  Policzyć liczbę krawędzi i wierzchołków.

In [1]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.col

Intitializing Scala interpreter ...

Spark Web UI available at http://2ba31a446ddf:4040
SparkContext available as 'sc' (version = 3.2.0, master = local[*], app id = local-1638976742312)
SparkSession available as 'spark'


import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.col


In [2]:
val edgesDF = spark.read.format("csv")
.option("header", "true")
.option("inferSchema", "true")
.load("musae_facebook_edges.csv")
edgesDF.show()


+----+-----+
|id_1| id_2|
+----+-----+
|   0|18427|
|   1|21708|
|   1|22208|
|   1|22171|
|   1| 6829|
|   1|16590|
|   1|20135|
|   1| 8894|
|   1|15785|
|   1|10281|
|   1|22265|
|   1| 7136|
|   1|22405|
|   1|10379|
|   1|13737|
|   1| 8533|
|   1|14344|
|   1| 2812|
|   1| 5755|
|   1|16260|
+----+-----+
only showing top 20 rows



edgesDF: org.apache.spark.sql.DataFrame = [id_1: int, id_2: int]


In [3]:
val verticesDF = spark.read.format("csv")
.option("header", "true")
.option("inferSchema", "true")
.load("musae_facebook_target.csv")

val vert_df = verticesDF.select("page_name","page_type" )
vert_df.show()


+--------------------+----------+
|           page_name| page_type|
+--------------------+----------+
|The Voice of Chin...|    tvshow|
|U.S. Consulate Ge...|government|
|                ESET|   company|
|Consulate General...|government|
|Mark Bailey MP - ...|politician|
| Victor Dominello MP|politician|
|Jean-Claude Poissant|politician|
|Deputado Ademir C...|politician|
|T.C. Mezar-ı Şeri...|government|
|Army ROTC Fightin...|government|
| NASA Student Launch|government|
|       Eliziane Gama|politician|
|     Socialstyrelsen|government|
|Brisbane Water LA...|government|
|NASA's Marshall S...|government|
|Municipio de Loma...|government|
|  Die Techniker (TK)|   company|
|     Digvijaya Singh|politician|
|1st Armored Divis...|government|
|           Shapeways|   company|
+--------------------+----------+
only showing top 20 rows



verticesDF: org.apache.spark.sql.DataFrame = [id: int, facebook_id: bigint ... 2 more fields]
vert_df: org.apache.spark.sql.DataFrame = [page_name: string, page_type: string]


In [4]:
val Vertices: RDD[(VertexId, (String, String))] = verticesDF
  .select(col("id").cast("long"), col("page_name"), col("page_type"))
  .rdd
  .map(row => (row.getLong(0), (row.getString(1), row.getString(2))))

Vertices: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, (String, String))] = MapPartitionsRDD[33] at map at <console>:32


In [5]:
val Edges: RDD[Edge[Long]] = edgesDF
  .select(col("id_1").cast("long"), col("id_2").cast("long"))
  .rdd
  .map(row => Edge(row.getLong(0), row.getLong(1), 1))

Edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Long]] = MapPartitionsRDD[40] at map at <console>:32


In [6]:
val defaultV = ("NameMissing", "TypeMissing")
val myGraph = Graph(Vertices, Edges, defaultV)

defaultV: (String, String) = (NameMissing,TypeMissing)
myGraph: org.apache.spark.graphx.Graph[(String, String),Long] = org.apache.spark.graphx.impl.GraphImpl@2aa5d05d


In [7]:
println("Total Number of Vertices: " + myGraph.numVertices)
println("Total Number of Edges: " + myGraph.numEdges)

Total Number of Vertices: 22470
Total Number of Edges: 171002


### Sprawdzić czy graf jest spójny. Czy dwa podgrafy utworzone dla typów strony politician oraz company też są spójne? 

In [8]:
myGraph.vertices.count() == myGraph.connectedComponents().vertices.count()

res3: Boolean = true


In [9]:
val my_subgraph1 = myGraph.subgraph(vpred = (id, attr) => attr._2 == "politician")
my_subgraph1.vertices.count() == my_subgraph1.connectedComponents().vertices.count()

my_subgraph1: org.apache.spark.graphx.Graph[(String, String),Long] = org.apache.spark.graphx.impl.GraphImpl@125475f3
res4: Boolean = true


In [10]:
val my_subgraph2 = myGraph.subgraph(vpred = (id, attr) => attr._2 == "company")
my_subgraph2.vertices.count() == my_subgraph2.connectedComponents().vertices.count()

my_subgraph2: org.apache.spark.graphx.Graph[(String, String),Long] = org.apache.spark.graphx.impl.GraphImpl@ffbdeaa
res5: Boolean = true


### Spośród 1000 stron o najwyższym PageRank znaleźć 50 takich (wypisać page_name i page_type), które mają najmniej połączeń oraz 50 o największej liczbie połączeń. Który typ strony był dominujący w każdej z tych kategorii? Narysować wykres zależności PageRank od liczby krawędzi dla wierzchołków (scatter plot)

In [11]:
val ranks = myGraph.pageRank(0.0001).vertices

ranks: org.apache.spark.graphx.VertexRDD[Double] = VertexRDDImpl[1844] at RDD at VertexRDD.scala:57


In [14]:
ranks

res8: org.apache.spark.graphx.VertexRDD[Double] = VertexRDDImpl[1844] at RDD at VertexRDD.scala:57


### Korzystając z Pregel API zaimplementować następujący algorytm. W pierwszym kroku wybrana strona publikuje post fake news. W kolejnym kroku ten post publikowany jest przez ⅓ losowo wybranych kontaktów tej strony. W dalszych krokach, dla każdej strony, która opublikowała już ten post, losowo wybrane ⅓ jej kontaktów publikuje go u siebie. Pokazać jak zmienia się liczba stron które opublikowały post w zależności od liczby kroków. (2 p.)
### Opcjonalnie: Sprawdzić powyższą zależność dla współczynnika innego niż 1/3